# Use  [siphon](https://unidata.github.io/siphon/latest/examples/upperair/Wyoming_Request.html#sphx-glr-examples-upperair-wyoming-request-py) to check whether [Wyoming Upper Air sounding](https://weather.uwyo.edu/upperair/sounding.html) data exists on specified dates


Sounding data interface:
https://weather.uwyo.edu/upperair/sounding.html

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 12})
import pandas as pd
from datetime import datetime
from datetime import timedelta

# meteorology - specific
from siphon.simplewebservice.wyoming import WyomingUpperAir

import sys
sys.path.append('./scripts/')

from LIB_dates import generate_date_list
from LIB_request_WUA_sounding import check_for_WUA_soundings

### Create list of dates to check

In [2]:
#===============================
# initial date
date_i = datetime(2020, 1, 1, 0)
# final date
date_f = datetime(2020, 12, 31, 23)
# time step in hours
hourly = 6
#===============================

# homemade function
# use help(generate_date_list) for details
dates = generate_date_list(date_i = date_i, date_f = date_f, hourly = hourly, suppress_prints = False)

 >>> list of length 1464 generated with 6-hourly steps between 2020-01-01 00:00:00 and 2020-12-31 18:00:00
 >>> with provided values, final date 2020-12-31 18:00:00 does not match provided date_f


### Use siphon to check whether sounding data exists on given dates
save dates to pandas data frame with sounding_exist == 1 when data exists and sounding_exist == 0 when no data found

In [ ]:
starttime = datetime.now()
print(f'anticipated runtime: {(len(dates))/60} minutes')

# homemade function
# use help(check_for_WUA_soundings) for details
df = check_for_WUA_soundings(date_list = dates, station = 'PABR', suppress_prints=False)

print(df)

print(f'\n\nruntime: {datetime.now()-starttime}')

anticipated runtime: 24.4 minutes


### Save list of dates to dataframe for later reference
save dates to pandas data frame with sounding_exist == 1 when data exists and sounding_exist == 0 when no data found

In [ ]:
# create name of csv file from date range
date_i_string = date_i.strftime('%Y%m%dZ%H')
date_f_string = date_f.strftime('%Y%m%dZ%H')
save_name = f'./sounding_checks/checked_{date_i_string}_{date_f_string}_hourly{hourly}.csv'

# save file
df.to_csv(path_or_buf=save_name, index=None)
print(f' >>> saving file as {save_name}')

pd.read_csv(save_name)